In [3]:
from rdkit import Chem
import numpy as np

mol = Chem.MolFromPDBFile('xyz_files/homopoly_3248_chain_2.pdb')

adj_matrix = Chem.GetAdjacencyMatrix(mol)
adj_matrix = np.array(adj_matrix)

print(adj_matrix)
print(adj_matrix.shape)

[[0 1 0 ... 0 0 0]
 [1 0 1 ... 0 0 0]
 [0 1 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 1 0]
 [0 0 0 ... 1 0 1]
 [0 0 0 ... 0 1 0]]
(56, 56)


In [5]:
n_atoms = mol.GetNumAtoms()
feature_matrix = np.zeros((n_atoms, 2))

for i, atom in enumerate(mol.GetAtoms()):
    feature_matrix[i, 0] = atom.GetAtomicNum()
    feature_matrix[i, 1] = atom.GetDegree()

print(feature_matrix)

[[6. 1.]
 [8. 2.]
 [6. 3.]
 [8. 1.]
 [6. 3.]
 [6. 2.]
 [6. 3.]
 [6. 2.]
 [6. 3.]
 [8. 1.]
 [6. 2.]
 [6. 2.]
 [6. 3.]
 [8. 1.]
 [6. 1.]
 [8. 2.]
 [6. 3.]
 [8. 1.]
 [6. 3.]
 [6. 3.]
 [6. 3.]
 [6. 2.]
 [6. 3.]
 [8. 1.]
 [6. 2.]
 [6. 2.]
 [6. 3.]
 [8. 1.]
 [6. 1.]
 [8. 2.]
 [6. 3.]
 [8. 1.]
 [6. 3.]
 [6. 3.]
 [6. 3.]
 [6. 2.]
 [6. 3.]
 [8. 1.]
 [6. 2.]
 [6. 2.]
 [6. 3.]
 [8. 1.]
 [6. 1.]
 [8. 2.]
 [6. 3.]
 [8. 1.]
 [6. 3.]
 [6. 2.]
 [6. 3.]
 [6. 2.]
 [6. 3.]
 [8. 1.]
 [6. 2.]
 [6. 2.]
 [6. 3.]
 [8. 1.]]
